# How to build advanced Chatbot with session memory using LangChain
* Advanced Chatbot LLM App
    * Will be able to have conversation
    * Will remember previous interactions: will have memory
    * Will be able to have different memories for different user sessions
    * Will be able to remember limited number of messages: limited memory

## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages
    *  LLM Model is based around raw text
* Chat History: allows Chat Model to remember previous interactions

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
MODEL_GPT = 'gpt-4o-mini'

In [3]:
# !pip list | grep langchain
!pip list | findstr langchain

langchain                                0.3.13
langchain-chroma                         0.1.4
langchain-community                      0.3.13
langchain-core                           0.3.43
langchain-experimental                   0.3.4
langchain-google-community               2.0.7
langchain-groq                           0.2.5
langchain-openai                         0.2.14
langchain-text-splitters                 0.3.4
langchainhub                             0.1.21


## Connect with LLM and start conversation with it

In [4]:
from langchain_openai import ChatOpenAI

# chatbot = ChatOpenAI(model="gpt-3.5-turbo")
chatbot = ChatOpenAI(model=MODEL_GPT)

In [5]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favourite color is blue."),
]

### Call ChatModel (LLM)

In [6]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="That's great! Blue is often associated with calmness, trust, and serenity. Do you have a particular shade of blue that you like the most, or any specific items that are blue that you enjoy?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 13, 'total_tokens': 55, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-08730cfc-b819-4c94-81e4-b66ab243bb77-0', usage_metadata={'input_tokens': 13, 'output_tokens': 42, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Track operation in LangSmith
* [Open LangSmith here](https://smith.langchain.com/)

## Check if Chatbot remembers your favorite color

In [7]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content='I don’t have access to personal information about you, so I can’t determine your favorite color. However, if you tell me what it is, I’d be happy to chat about it!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 13, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-e63b79ad-ce84-4f90-a4cc-5b9360cbb6ae-0', usage_metadata={'input_tokens': 13, 'output_tokens': 40, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Add memory to Chatbot
* We will use ChatMessageHistory package
* We will save Chatbot memory in python dictionary called chatbotMemory
* We will define get_session_history function to create session_id for each conversation

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chatbotMemory = {}

# input: session_id, output: chatbotMemory[session_id]
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] = ChatMessageHistory()
    return chatbotMemory[session_id]


chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot, 
    get_session_history
)

In [9]:
session1 = {"configurable": {"session_id": "001"}}

In [10]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite color is red.")],
    config=session1,
)

responseFromChatbot.content

"That's great! Red is a vibrant and strong color often associated with energy, passion, and excitement. Do you have a favorite shade of red, or is there a particular reason you like it?"

In [11]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red!'

## Change session and see what happens

In [12]:
session2 = {"configurable": {"session_id": "002"}}

In [13]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session2,
)

responseFromChatbot.content

"I don't know your favorite color. If you'd like to share it, I'd love to hear!"

## Go back to session1 and see if memory is still there

In [14]:
session1 = {"configurable": {"session_id": "001"}}

In [15]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red!'

## ChatBot has session memory now. Check if it remembers conversation from session2

In [16]:
session2 = {"configurable": {"session_id": "002"}}

In [17]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mi name is Julio.")],
    config=session2,
)

responseFromChatbot.content

"Nice to meet you, Julio! Do you have a favorite color you'd like to share?"

In [18]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=session2,
)

responseFromChatbot.content

'Your name is Julio.'

In [19]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red.'

## ChatBot now remembers each of conversations

## Importance to manage Conversation History

In [20]:
print(chatbotMemory)

{'001': InMemoryChatMessageHistory(messages=[HumanMessage(content='My favorite color is red.', additional_kwargs={}, response_metadata={}), AIMessage(content="That's great! Red is a vibrant and strong color often associated with energy, passion, and excitement. Do you have a favorite shade of red, or is there a particular reason you like it?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 13, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-c7358f73-6ed8-46f4-a9ff-44f56962dbf4-0', usage_metadata={'input_tokens': 13, 'output_tokens': 40, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough


def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt 
    | chatbot
)

In [22]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages",
)

### Add 2 more messages to session1 conversation

In [23]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite vehicles are Vespa scooters.")],
    config=session1,
)

responseFromChatbot.content

'Vespa scooters are iconic! They have a classic design and are known for their style and ease of use. Do you have a favorite model or color of Vespa?'

In [24]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite city is San Francisco.")],
    config=session1,
)

responseFromChatbot.content

"San Francisco is a fantastic choice! It's known for its beautiful scenery, iconic landmarks like the Golden Gate Bridge, vibrant culture, and diverse neighborhoods. Do you have a favorite spot or activity in San Francisco?"

## Chatbot memory has now 4 messages. Check Chatbot with limited memory

In [25]:
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my favorite color?")],
    },
    config=session1,
)

responseFromChatbot.content

"I don't have access to personal information about you, so I can't determine your favorite color. However, if you share it with me, I'd be happy to discuss it further!"

## Compare previous response with one provided by Chatbot with unlimited memory

In [26]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="what is my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Your favorite color is red!'